In [154]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import re

In [155]:
data = pd.read_json('../Parcer/Cleaning/data.json')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   product_id               133 non-null    object 
 1   type                     131 non-null    object 
 2   grace_period             131 non-null    object 
 3   credit_limit             131 non-null    object 
 4   first_year_fee           131 non-null    object 
 5   second_year_fee          122 non-null    object 
 6   min_age                  130 non-null    object 
 7   max_age                  130 non-null    object 
 8   citizenship              130 non-null    object 
 9   registration             130 non-null    object 
 10  work_experience          130 non-null    object 
 11  maximum                  131 non-null    object 
 12  repayment_method         131 non-null    object 
 13  max_monthly_cashback     131 non-null    object 
 14  ndfl_2_3_4               1

In [156]:
data = data.drop(columns=['product_name', 'bank', 'product_id'])

In [157]:
cat_col = ['bank_certificate', 'ndfl_2_3_4', 'other_documents', 'pension_certificate',
           'free_form_certificate', 'additional_income_proof', 'payment_system',
           'registration', 'citizenship']
for i in cat_col:
    print(data[i].value_counts(), end='\n\n')

bank_certificate
нет требуется                                                              91
по выбору, если требуется один или несколько документов из одной группы    35
обязательно                                                                 3
желательно или по требованию                                                2
Name: count, dtype: int64

ndfl_2_3_4
нет требуется                                                              61
по выбору, если требуется один или несколько документов из одной группы    44
обязательно                                                                20
желательно или по требованию                                                6
Name: count, dtype: int64

other_documents
нет требуется                                                              122
по выбору, если требуется один или несколько документов из одной группы      6
обязательно                                                                  2
желательно или по требованию             

In [158]:

onehotencoder = OneHotEncoder(sparse_output=False)
data_ohe = onehotencoder.fit_transform(data[cat_col])
one_hot_df = pd.DataFrame(data_ohe, columns=onehotencoder.get_feature_names_out(cat_col))
one_hot_df

,bank_certificate_желательно или по требованию,bank_certificate_нет требуется,bank_certificate_обязательно,"bank_certificate_по выбору, если требуется один или несколько документов из одной группы",bank_certificate_None,ndfl_2_3_4_желательно или по требованию,ndfl_2_3_4_нет требуется,ndfl_2_3_4_обязательно,"ndfl_2_3_4_по выбору, если требуется один или несколько документов из одной группы",ndfl_2_3_4_None,...,"registration_временная, постоянная","registration_временная, постоянная в любом регионе присутствия банка",registration_постоянная,registration_постоянная в любом регионе присутствия банка,registration_постоянная в регионе обращения,"registration_постоянная в регионе обращения, постоянная в любом регионе присутствия банка",registration_None,citizenship_не требуется,citizenship_требуется,citizenship_None
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
129,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
130,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
131,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [159]:
condition = (
    (one_hot_df['bank_certificate_None'] == 1.0) |
    (one_hot_df['ndfl_2_3_4_None'] == 1.0) |
    (one_hot_df['registration_None'] == 1.0) |
    (one_hot_df['citizenship_None'] == 1.0) |
    (one_hot_df['additional_income_proof_None'] == 1.0) | 
    (one_hot_df['pension_certificate_None'] == 1.0) |
    (one_hot_df['other_documents_None'] == 1.0)
)
one_hot_df = one_hot_df[~condition]
one_hot_df = one_hot_df.drop(columns=['payment_system_None', 'citizenship_None', 
                         'additional_income_proof_None', 'pension_certificate_None',
                         'other_documents_None', 'ndfl_2_3_4_None', 'bank_certificate_None',
                         'registration_None', 'free_form_certificate_None'])

In [160]:
data = data.drop(columns=cat_col)
data_encoded = pd.concat([data, one_hot_df], axis=1)
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 55 columns):
 #   Column                                                                                           Non-Null Count  Dtype  
---  ------                                                                                           --------------  -----  
 0   type                                                                                             131 non-null    object 
 1   grace_period                                                                                     131 non-null    object 
 2   credit_limit                                                                                     131 non-null    object 
 3   first_year_fee                                                                                   131 non-null    object 
 4   second_year_fee                                                                                  122 non-null    object 
 5   min_age 

In [161]:
data_encoded['grace_period'].value_counts()

grace_period
55 дней        21
50 дней        16
62 дня         14
120 дней       13
100 дней        6
115 дней        5
110 дней        4
до 3 лет        4
180 дней        4
51 день         3
до 60 дней      3
60 дней         3
56 дней         3
Нет             2
150 дней        2
112 дней        2
3 года          2
130 дней        2
90 дней         2
до 62 дней      2
123 дня         1
1094 дня        1
40 дней         1
от 121 дня      1
61 день         1
до 56 дней      1
до 117 дней     1
755 дней        1
111 дней        1
до 115 дней     1
до 110 дней     1
415 дней        1
4 месяца        1
до 50 дней      1
212 дней        1
200 дней        1
2 месяца        1
30 дней         1
Name: count, dtype: int64

In [162]:
def convert_to_days(value):
    if isinstance(value, str):
        if "лет" in value or "года" in value:
            years = re.findall(r'\d+', value)
            if years:
                return float(years[0]) * 365
        
        elif "месяц" in value:
            months = re.findall(r'\d+', value)
            if months:
                return float(months[0]) * 30
        
        elif "до" in value:
            days = re.findall(r'\d+', value)
            if days:
                return float(days[0])
        
        elif "от" in value:
            days = re.findall(r'\d+', value)
            if days:
                return float(days[0])
        
        elif "день" in value or "дня" in value or "дней" in value:
            days = re.findall(r'\d+', value)
            if days:
                return float(days[0])
        
        else:
            return None
    
    return value


data_encoded['grace_period'] = data_encoded['grace_period'].apply(convert_to_days)

In [163]:
def split_and_clean(value):
    if pd.isna(value) or value in ['', ' ']:
        return []
    
    parts = re.split(r',', value)
    result = []
    
    for part in parts:

        if '/' in part:
            subparts = re.split(r'/', part)
            main_service = subparts[0].strip()
            for subpart in subparts:
                result.append(f"{main_service} {subpart.strip()}")
        else:
            result.append(part.strip())
    
    return result


data_encoded['type_split'] = data_encoded['type'].apply(split_and_clean)

all_categories = set()
for categories in data_encoded['type_split']:
    all_categories.update(categories)

for category in sorted(all_categories):
    data_encoded[category] = data_encoded['type_split'].apply(lambda x: 1 if category in x else 0)

data_encoded.drop(columns=['type_split', 'type'], inplace=True)


In [165]:
def parse_credit_limit(value):
    if pd.isna(value):
        return None, None
    
    value = value.replace("₽", "").replace(",", "").strip()
    
    lower_bound = None
    upper_bound = None
    
    if "до" in value and "от" not in value:
        match = re.search(r"до\s+(\d+)", value)
        if match:
            upper_bound = int(match.group(1).replace(" ", ""))
    
    elif "от" in value and "до" in value:
        match = re.search(r"от\s+(\d+)\s+до\s+(\d+)", value)
        if match:
            lower_bound = int(match.group(1).replace(" ", ""))
            upper_bound = int(match.group(2).replace(" ", ""))
    
    elif "от" in value and "до" not in value:
        match = re.search(r"от\s+(\d+)", value)
        if match:
            lower_bound = int(match.group(1).replace(" ", ""))
    
    elif value.isdigit():
        upper_bound = int(value.replace(" ", ""))
    
    elif "любой" in value:
        lower_bound = 0
        upper_bound = float("inf")
    
    return lower_bound, upper_bound


data_encoded[['below_b', 'upper_b']] = data_encoded['credit_limit'].apply(lambda x: pd.Series(parse_credit_limit(x)))
data_encoded.drop(columns='credit_limit', inplace=True)

In [166]:
data_encoded['min_age'] = data_encoded['min_age'].dropna().apply(lambda x: x[:2])
data_encoded['max_age'] = data_encoded['max_age'].dropna().apply(lambda x: x[:2])
print(data_encoded['min_age'].value_counts())
print(data_encoded['max_age'].value_counts())

min_age
18    58
21    42
20    18
19     8
23     3
24     1
Name: count, dtype: int64
max_age
70    67
65    21
62    12
75    11
60     5
73     3
66     3
67     2
69     1
68     1
58     1
63     1
23     1
57     1
Name: count, dtype: int64
